## Find Best Locations in Melbourne to open Goila Butter Chicken's Cloud Kitchen

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Extract Melbourne Neighborhoods and Get Location Coordinates</a>
    - Scrape Wikipedia page to get the Melbourne City Neighborhoods
    - Use Nominatim OpenStreet site to get coordinates for Melbourne and all its neighborhoods found above
    - Visualize the neighborhoods using Folium
    
      
2.  <a href="#item1">Explore and Cluster the Neighborhoods</a>
    - Use Foursquare API to get nearby venues and venue categories.  
    - Cluster the neighborhoods based on venue categories, using K-means algorithm 
    - Visualize the clustered neighborhoods using Folium
    - Get the top 10 most common venue categories in each neighborhood
    - Examine each cluster and determine how they are distinguished
  
  
3.  <a href="#item1">Recommend Locations to open Cloud Kitchen</a>  
    - Find top 5 neighborhoods with Indian Restaurants
    - Find the neighborhoods with Indian Restaurants amongst Top 10 venues
    - Observations and Conclusion

<a href="#item2"></a>

</font>
</div>

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans

## Extract Melbourne Neighborhoods and Get Location Coordinates

####  Scrape Wikipedia page to get the Melbourne City Neighborhoods

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs#City_of_Melbourne'

response = requests.get(url)
content = response.content
data = BeautifulSoup(content, 'html.parser')

In [3]:
suburbs = []
for link in data.find_all('a',title=True):
    suburbs.append(link.get('title'))

In [17]:
melbourne_neighborhoods = pd.DataFrame(suburbs)
melbourne_neighborhoods.head()

,0
0,Suburbs and localities (Australia)
1,Metropolitan area
2,Melbourne
3,Victoria (Australia)
4,Australia


In [18]:
# Filter rows belonging to Melbourne City as shown on wikipedia page
melbourne_neighborhoods = melbourne_neighborhoods.iloc[16:38]
melbourne_neighborhoods.head()

,0
16,"Carlton, Victoria"
17,"Carlton North, Victoria"
18,"Docklands, Victoria"
19,"East Melbourne, Victoria"
20,"Jolimont, District of Melbourne"


In [20]:
# Remove rows which do not have postcode on wikipedia page
melbourne_neighborhoods.drop([20,21,25,26,27,29,31,37], inplace=True)

In [21]:
melbourne_neighborhoods.reset_index(inplace=True, drop=True)
melbourne_neighborhoods.columns = ['Neighborhood']
melbourne_neighborhoods.head()

,Neighborhood
0,"Carlton, Victoria"
1,"Carlton North, Victoria"
2,"Docklands, Victoria"
3,"East Melbourne, Victoria"
4,"Flemington, Victoria"


In [22]:
# Checking for unexpected values in Station
melbourne_neighborhoods['Neighborhood'].unique()

array(['Carlton, Victoria', 'Carlton North, Victoria',
       'Docklands, Victoria', 'East Melbourne, Victoria',
       'Flemington, Victoria', 'Kensington, Victoria',
       'Melbourne City Centre', 'North Melbourne, Victoria',
       'Parkville, Victoria', 'Port Melbourne, Victoria',
       'Southbank, Victoria', 'South Wharf, Victoria',
       'South Yarra, Victoria', 'West Melbourne, Victoria'], dtype=object)

In [23]:
# Names are appended with state name (Victoria), which we don't need and so will remove
melbourne_neighborhoods['Neighborhood'] = melbourne_neighborhoods['Neighborhood'].str.split(',').str[0]
melbourne_neighborhoods['Neighborhood'].unique()

array(['Carlton', 'Carlton North', 'Docklands', 'East Melbourne',
       'Flemington', 'Kensington', 'Melbourne City Centre',
       'North Melbourne', 'Parkville', 'Port Melbourne', 'Southbank',
       'South Wharf', 'South Yarra', 'West Melbourne'], dtype=object)

#### Use Nominatim OpenStreet site to get coordinates for London and all its neighborhoods found above

In [24]:
address = 'Melbourne City, Victoria, Australia'

url = 'https://nominatim.openstreetmap.org/search?q={}&format=jsonv2'.format(address)
response = requests.get(url)
content = response.json()

latitude = float(content[0]['lat'])
longitude = float(content[0]['lon'])
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [25]:
lat = []
lon = []
for neighborhood in melbourne_neighborhoods['Neighborhood']:
    address = neighborhood + ', Victoria, Australia'
    url = 'https://nominatim.openstreetmap.org/search?q={}&format=jsonv2'.format(address)
    response = requests.get(url)
    content = response.json()
    lat.append(float(content[0]['lat']))
    lon.append(float(content[0]['lon']))

melbourne_neighborhoods['Latitude'] = lat
melbourne_neighborhoods['Longitude'] = lon

In [26]:
melbourne_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Carlton,-37.800144,144.969268
1,Carlton North,-37.784559,144.972855
2,Docklands,-37.817870,144.938654
3,East Melbourne,-37.815588,144.982555
4,Flemington,-37.787277,144.923242


In [27]:
melbourne_neighborhoods.shape

(14, 3)

#### Visualize the neighborhoods using Folium

In [29]:
# create map of New York using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, area in zip(melbourne_neighborhoods['Latitude'], melbourne_neighborhoods['Longitude'], melbourne_neighborhoods['Neighborhood']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

Next, we will use the Foursquare API to explore the neighborhoods in above filtered boroughs and segment them.

## Explore and Cluster the Neighborhoods

#### Use Foursquare API to get nearby venues and venue categories

In [30]:
ID = '*****' # Foursquare ID
KEY = '*****' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # Foursquare API limit value

In [31]:
cols = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

neighborhood_venues = pd.DataFrame(columns = cols)

In [36]:
for columns, neighborhood in melbourne_neighborhoods.iterrows():
    radius = 2000
    neigborhood_name = neighborhood['Neighborhood']
    neighborhood_lat = neighborhood['Latitude']
    neighborhood_lon = neighborhood['Longitude']
        
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
    .format(ID, KEY, VERSION, neighborhood_lat, neighborhood_lon, radius, LIMIT)

    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    for v in venues:
        neighborhood_venues = neighborhood_venues.append({'Neighborhood': neigborhood_name, 
                                              'Neighborhood Latitude': neighborhood_lat,
                                              'Neighborhood Longitude': neighborhood_lon,
                                              'Venue': v['venue']['name'],
                                              'Venue Latitude': v['venue']['location']['lat'],
                                              'Venue Longitude': v['venue']['location']['lng'],
                                              'Venue Category': v['venue']['categories'][0]['name']}, ignore_index=True)

neighborhood_venues.head()            

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carlton,-37.800144,144.969268,Carlton Wine Room,-37.798584,144.968610,Wine Bar
1,Carlton,-37.800144,144.969268,Gewürzhaus,-37.799050,144.967480,Gourmet Shop
2,Carlton,-37.800144,144.969268,D.O.C. Pizza & Mozzarella Bar,-37.798954,144.968490,Pizza Place
3,Carlton,-37.800144,144.969268,Yo-Chi,-37.798659,144.967849,Frozen Yogurt Shop
4,Carlton,-37.800144,144.969268,Pidapipó Gelateria,-37.798300,144.967060,Ice Cream Shop


In [37]:
# Count of venues
neighborhood_venues.shape[0]

2423

In [38]:
# Count of unique venues
len(neighborhood_venues['Venue Category'].unique())

244

In [39]:
# Count of unique venue category in each neighborhood
neighborhood_venues.groupby('Neighborhood')['Venue Category'].count()

Neighborhood
Carlton                  200
Carlton North            197
Docklands                193
East Melbourne           200
Flemington               154
Kensington               153
Melbourne City Centre    200
North Melbourne          200
Parkville                141
Port Melbourne           102
South Wharf              200
South Yarra              200
Southbank                200
West Melbourne            83
Name: Venue Category, dtype: int64

#### Cluster the neighborhoods based on venue categories, using K-means algorithm

In [40]:
# one hot encoding
venues_encoded = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe at first position
venues_encoded.insert(0, 'Neighborhood', neighborhood_venues['Neighborhood'].values)

venues_encoded.head()

,Neighborhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Bookstore,Botanical Garden,Boutique,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Cambodian Restaurant,Candy Store,Cantonese Restaurant,Casino,Cemetery,Chaat Place,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Night Market,Nightclub,Noodle House,Opera House,Organic Grocery,Outlet Mall,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Road,Rock Club,Roof Deck,Rooftop Bar,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soba Restaurant,Social Club,Spanish Restaurant,Speakeasy,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,Carlton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Carlton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [41]:
venues_encoded.shape

(2423, 245)

In [42]:
venues_encoded = venues_encoded.groupby('Neighborhood').mean().reset_index()
venues_encoded.head()

,Neighborhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Bookstore,Botanical Garden,Boutique,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Cambodian Restaurant,Candy Store,Cantonese Restaurant,Casino,Cemetery,Chaat Place,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Night Market,Nightclub,Noodle House,Opera House,Organic Grocery,Outlet Mall,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Road,Rock Club,Roof Deck,Rooftop Bar,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soba Restaurant,Social Club,Spanish Restaurant,Speakeasy,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,Carlton,0.005000,0.0,0.0,0.0,0.000000,0.005,0.005,0.00,0.000,0.0,0.025000,0.000,0.010000,0.000000,0.00,0.000000,0.030000,0.060000,0.0,0.0,0.0,0.005,0.000000,0.020000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.005,0.000000,0.000000,0.010000,0.0,0.130000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.005000,0.005000,0.000000,0.000000,0.0,0.000000,0.035000,0.060000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.005,0.000,0.005,0.000000,0.030000,0.0,0.005000,0.0,0.000000,0.0,0.000000,0.005000,0.000000,0.0,0.005,0.000000,0.000000,0.000000,0.000000,0.000000,

In [43]:
venues_encoded.shape

(14, 245)

In [44]:
X = venues_encoded.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=3, random_state=3)
kmeans.fit(X)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_)

{0, 1, 2}

In [45]:
# add Cluster Labels to Neighborhoods Data 
melbourne_neighborhoods.insert(0, 'Cluster', kmeans.labels_).copy()
melbourne_neighborhoods.head()

,Cluster,Neighborhood,Latitude,Longitude
0,1,Carlton,-37.800144,144.969268
1,1,Carlton North,-37.784559,144.972855
2,2,Docklands,-37.817870,144.938654
3,1,East Melbourne,-37.815588,144.982555
4,0,Flemington,-37.787277,144.923242


#### Visualize the clustered neighborhoods using Folium

In [48]:
# create map of london
melbourne_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melbourne_neighborhoods['Latitude'], melbourne_neighborhoods['Longitude'], melbourne_neighborhoods['Neighborhood'], melbourne_neighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' :Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(melbourne_clusters_map)
       
melbourne_clusters_map

Now, continuing further we will find most common venues of in these neighborhoods and see what is common within each cluster

#### Get the top 10 most common venue categories in each neighborhood

In [49]:
cols = ['Neighborhood'] + ['Most Common Venue '+str(i) for i in range(1,11)]

neighborhood_top_venues = pd.DataFrame(columns=cols)

unique_neighborhoods = neighborhood_venues['Neighborhood'].unique()

for neighbor in unique_neighborhoods:
    df = neighborhood_venues[neighborhood_venues['Neighborhood'] == neighbor]
    top_venues = df['Venue Category'].value_counts().sort_values(ascending=False).head(10).index.to_list()
    
    if len(top_venues) >= 10:
        
        neighborhood_top_venues = neighborhood_top_venues.append({cols[0]:neighbor,
                                                              cols[1]:top_venues[0],
                                                              cols[2]:top_venues[1],
                                                              cols[3]:top_venues[2],
                                                              cols[4]:top_venues[3],
                                                              cols[5]:top_venues[4],
                                                              cols[6]:top_venues[5],
                                                              cols[7]:top_venues[6],
                                                              cols[8]:top_venues[7],
                                                              cols[9]:top_venues[8],
                                                              cols[10]:top_venues[9]}, ignore_index=True)
        
neighborhood_top_venues.head()

,Neighborhood,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,Carlton,Café,Bar,Italian Restaurant,Coffee Shop,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bakery,Deli / Bodega,Japanese Restaurant
1,Carlton North,Café,Bar,Bakery,Italian Restaurant,Pub,Wine Bar,Ice Cream Shop,Wine Shop,Vegetarian / Vegan Restaurant,Tram Station
2,Docklands,Café,Coffee Shop,Hotel,Restaurant,Korean Restaurant,Dessert Shop,Park,Japanese Restaurant,Burger Joint,Brewery
3,East Melbourne,Café,Coffee Shop,Park,Cricket Ground,Hotel,Cocktail Bar,Tennis Stadium,Japanese Restaurant,Wine Bar,Thai Restaurant
4,Flemington,Café,Pizza Place,Pub,Vietnamese Restaurant,Supermarket,Convenience Store,Grocery Store,Park,Malay Restaurant,Pharmacy


In [50]:
neighborhood_top_venues.shape

(14, 11)

#### Examine each cluster and determine how they are distinguished

In [52]:
melbourne_cluster_venues = pd.merge(left=melbourne_neighborhoods, right=neighborhood_top_venues, how='left', on='Neighborhood')
melbourne_cluster_venues.head()

,Cluster,Neighborhood,Latitude,Longitude,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,1,Carlton,-37.800144,144.969268,Café,Bar,Italian Restaurant,Coffee Shop,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bakery,Deli / Bodega,Japanese Restaurant
1,1,Carlton North,-37.784559,144.972855,Café,Bar,Bakery,Italian Restaurant,Pub,Wine Bar,Ice Cream Shop,Wine Shop,Vegetarian / Vegan Restaurant,Tram Station
2,2,Docklands,-37.817870,144.938654,Café,Coffee Shop,Hotel,Restaurant,Korean Restaurant,Dessert Shop,Park,Japanese Restaurant,Burger Joint,Brewery
3,1,East Melbourne,-37.815588,144.982555,Café,Coffee Shop,Park,Cricket Ground,Hotel,Cocktail Bar,Tennis Stadium,Japanese Restaurant,Wine Bar,Thai Restaurant
4,0,Flemington,-37.787277,144.923242,Café,Pizza Place,Pub,Vietnamese Restaurant,Supermarket,Convenience Store,Grocery Store,Park,Malay Restaurant,Pharmacy


In [53]:
# Cluster 0
melbourne_cluster_venues[melbourne_cluster_venues['Cluster'] == 0]

,Cluster,Neighborhood,Latitude,Longitude,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
4,0,Flemington,-37.787277,144.923242,Café,Pizza Place,Pub,Vietnamese Restaurant,Supermarket,Convenience Store,Grocery Store,Park,Malay Restaurant,Pharmacy
5,0,Kensington,-37.793938,144.930565,Café,Pub,Pizza Place,Park,Vietnamese Restaurant,Gym,Malay Restaurant,Supermarket,Racetrack,Grocery Store
8,0,Parkville,-37.787250,144.949596,Café,Zoo Exhibit,Park,Coffee Shop,Bar,Dog Run,Convenience Store,Bakery,Pub,Tram Station
9,0,Port Melbourne,-37.833361,144.921920,Café,Beach,Sandwich Place,Pub,Park,Gym,Go Kart Track,Fish & Chips Shop,Grocery Store,Breakfast Spot
13,0,West Melbourne,-37.810688,144.920404,Café,Dessert Shop,Sandwich Place,Park,Gym,Flea Market,Juice Bar,Food Truck,Performing Arts Venue,Ice Cream Shop


In [54]:
# Cluster 1
melbourne_cluster_venues[melbourne_cluster_venues['Cluster'] == 1]

,Cluster,Neighborhood,Latitude,Longitude,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
0,1,Carlton,-37.800144,144.969268,Café,Bar,Italian Restaurant,Coffee Shop,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bakery,Deli / Bodega,Japanese Restaurant
1,1,Carlton North,-37.784559,144.972855,Café,Bar,Bakery,Italian Restaurant,Pub,Wine Bar,Ice Cream Shop,Wine Shop,Vegetarian / Vegan Restaurant,Tram Station
3,1,East Melbourne,-37.815588,144.982555,Café,Coffee Shop,Park,Cricket Ground,Hotel,Cocktail Bar,Tennis Stadium,Japanese Restaurant,Wine Bar,Thai Restaurant
6,1,Melbourne City Centre,-37.814218,144.963161,Bar,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Whisky Bar,Shopping Mall,Hobby Shop,Bookstore,Korean Restaurant
12,1,South Yarra,-37.837770,144.991854,Café,French Restaurant,Coffee Shop,Bakery,Italian Restaurant,Japanese Restaurant,Hotel,Park,Pizza Place,Mexican Restaurant


In [55]:
# Cluster 2
melbourne_cluster_venues[melbourne_cluster_venues['Cluster'] == 2]

,Cluster,Neighborhood,Latitude,Longitude,Most Common Venue 1,Most Common Venue 2,Most Common Venue 3,Most Common Venue 4,Most Common Venue 5,Most Common Venue 6,Most Common Venue 7,Most Common Venue 8,Most Common Venue 9,Most Common Venue 10
2,2,Docklands,-37.817870,144.938654,Café,Coffee Shop,Hotel,Restaurant,Korean Restaurant,Dessert Shop,Park,Japanese Restaurant,Burger Joint,Brewery
7,2,North Melbourne,-37.807609,144.942351,Café,Coffee Shop,Italian Restaurant,Korean Restaurant,Dessert Shop,Chinese Restaurant,Bar,Burger Joint,Pub,Pizza Place
10,2,Southbank,-37.825392,144.963578,Café,Hotel,Bar,Coffee Shop,Japanese Restaurant,Park,Cocktail Bar,Theater,Burger Joint,Plaza
11,2,South Wharf,-37.825344,144.952395,Café,Coffee Shop,Japanese Restaurant,Hotel,Burger Joint,Bar,Seafood Restaurant,Gastropub,Cocktail Bar,Australian Restaurant


Cluster 0: Neighborhoods to the west of centre with Cafes and Fast Food joints. Also, has additional variety in terms of markets, stores, gyms, parks.
  
Cluster 1: Neighborhoods to the east of centre with Cafes, Coffee Shops, Bars and Restaurants. Seem to offer lots of wine and dine options. Also has City Centre in it, so sure to be buzzing with acitivity.
  
Cluster 2: Neighborhoods near to center with Cafes, Coffee Shops and lots of Asian Restaurants.

It's difficult to zero in on any one cluster as all seem to have great neighborhoods offering variety of options. Looking at the map, we can shortlist a few locations around city centre but not too far away. Also, we will avoid going south and keep in the vicinity of east, north and west so we can look to service mutliple areas.
  
Neighborhoods - **Kensignton, Carlton, North Melbourne, East Melbourne and Docklands** seem to be good choices.

## Recommend Locations to open Cloud Kitchen

#### Find top 5 neighborhoods with Indian Restaurants

In [59]:
neighborhood_venues_indian = neighborhood_venues[neighborhood_venues['Venue Category'].str.contains('Indian')]
neighborhood_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
113,Carlton North,-37.784559,144.972855,Horn Please,-37.785203,144.982174,Indian Restaurant
155,Carlton North,-37.784559,144.972855,Singhs Gourmet Indian Foods,-37.778410,144.978337,Indian Restaurant
215,Docklands,-37.817870,144.938654,Bhoj Indian Restaurant,-37.815072,144.941499,Indian Restaurant
281,Docklands,-37.817870,144.938654,Shiraaz Indian,-37.814603,144.943576,Indian Restaurant
331,East Melbourne,-37.815588,144.982555,Tonka,-37.815749,144.971313,Indian Restaurant


In [60]:
neighborhood_venues_indian['Neighborhood'].value_counts().head()

Carlton North      3
East Melbourne     3
Docklands          3
North Melbourne    3
Parkville          1
Name: Neighborhood, dtype: int64

#### Find the neighborhoods with Indian Restaurants amongst Top 10 venues

In [61]:
# Get rows from top venues dataframe where 'Indian Restaurant' is one of the venues
neighborhood_top_venues_indian = melbourne_cluster_venues[melbourne_cluster_venues.isin(['Indian Restaurant'])]

In [62]:
# Get index of rows as found above
neighborhood_top_venues_indian[neighborhood_top_venues_indian.notnull().any(axis=1)].index

Int64Index([], dtype='int64')

So, we do not have any Indian Restaurants among the top 10 venues of any neighborhood.

#### Observations and Conclusion

- Based on cluster analysis, we shortlisted Kensignton, Carlton, North Melbourne, East Melbourne and Docklands.  
  
  
- Top 4 neighborhoods with Indian Restaurants are Carlton North, East Melbourne, North Melbourne and Docklands. So, 3 are same as we selected above.  
  
  
- Looking at the map, we also notice that **Carlton** lies almost at the middle of triangle formed by North Melbourne, East Melbourne and Carlton North, and can service Carlton North and East Melbourne areas  
  
  
- Similarly, North Melbourne and Docklands can service areas towards west and south.
  
  
Based on these observations, we would recommend one of **Carlton, North Melbourne or Docklands** as the locations for opening our new **'Goila Butter Chicken**' Cloud Kitchen.